In [35]:
import pandas as pd
import gzip
import numpy as np

In [27]:
LOCAL = 'Lausanne'

In [28]:
# Specify the file paths
activity_file = 'data_original/vaud_activities.csv.gz'
population_file = 'data_original/vaud_population.csv.gz'
trip_file = 'data_original/vaud_trips.csv.gz'

# Read the gzipped CSV files
def read_gzipped_csv(file_path):
    with gzip.open(file_path, 'rt') as file:
        df = pd.read_csv(file)
    return df

# Read the dataframes
activity_vaud = read_gzipped_csv(activity_file)
population_vaud = read_gzipped_csv(population_file)[['id', 'age', 'home_x', 'home_y', 'local']].drop_duplicates()
trip_vaud = read_gzipped_csv(trip_file)[['Unnamed: 0', 'id', 'mode', 'dep_time','trav_time','start_link','end_link']].drop_duplicates()

print("File opened and read \n")

File opened and read 



In [29]:
population_local = population_vaud[population_vaud['local'] == LOCAL]
population_local_ids = population_local['id'].unique()

population_local.head(1)

,id,age,home_x,home_y,local
34200,6336880,30,2539957.0,1152783.0,Lausanne


In [42]:
trip_local = trip_vaud[trip_vaud['id'].isin(population_local_ids)]

trip_local.head(1)

,Unnamed: 0,id,mode,dep_time,trav_time,start_link,end_link
147102,25983202,6336880,car,07:06:35,00:10:06,290992,112618


In [31]:
activity_local = activity_vaud[activity_vaud['id'].isin(population_local_ids)]
count_act_by_types = activity_local.groupby('type')['facility'].nunique().reset_index()
print(f"Here's the count of facilities by types in {LOCAL} : {count_act_by_types}")
activity_local_filtered = activity_local[~activity_local['type'].isin(['other', 'pt interaction'])]

Here's the count of facilities by types in Lausanne :              type  facility
0       education       220
1            home     70975
2         leisure      6277
3           other     10178
4  pt interaction         0
5            shop      3177
6            work     13173


In [32]:
activity_local_filtered.head(1)

,Unnamed: 0,id,type,facility,link,x,y,start_time,end_time
178740,31913071,6336880,home,home2791489,290992,2539957.0,1152783.0,NaN,07:06:35


In [43]:
print(activity_local_filtered['Unnamed: 0'].nunique()) # nb de lignes
print(activity_local_filtered['id'].nunique()) # nb d'habitants
print(activity_local_filtered['link'].nunique()) # nb de link
print(activity_local_filtered['facility'].nunique()) # nb de facility
# il a 92k facilities pour 15k link : plusieurs facility a un link
# pourtnat les link sont pas forcement les memes coord mais bon a chaque fois tres proche
# il faut que je fasse un dataframe des liens avec leur mean coord ou juste la 1ere apparition (groupby et mean)
# puis ensuite sur ca je fais un merge
link_df = activity_local_filtered[['link', 'x', 'y']]
link_df = link_df.groupby('link').mean()
link_df.head(1)

538099
139392
11540
92245


,x,y
link,,
1000167,2528786.0,1161402.0


In [47]:
# merge with start_link
trip_local_merged = pd.merge(trip_local, link_df, left_on='start_link', right_index=True)
trip_local_merged.rename(columns={'x': 'start_x', 'y': 'start_y'}, inplace=True)

# merge with end_link
trip_local_merged = pd.merge(trip_local_merged, link_df, left_on='end_link', right_index=True)
trip_local_merged.rename(columns={'x': 'end_x', 'y': 'end_y'}, inplace=True)

trip_local_merged.head()

# pour chaque tripreer une nouvelle colonne avcec direct la distance calcule from le df des activities

,Unnamed: 0,id,mode,dep_time,trav_time,start_link,end_link,start_x,start_y,end_x,end_y
147102,25983202,6336880,car,07:06:35,00:10:06,290992,112618,2.539915e+06,1.152774e+06,2.538477e+06,1.152417e+06
1880518,27935067,6740390,walk,08:35:31,00:26:13,290992,112618,2.539915e+06,1.152774e+06,2.538477e+06,1.152417e+06
147103,25983203,6336880,car,16:51:35,00:00:00,112618,112618,2.538477e+06,1.152417e+06,2.538477e+06,1.152417e+06
149052,25985356,6337295,car,08:38:07,00:00:00,112618,112618,2.538477e+06,1.152417e+06,2.538477e+06,1.152417e+06
149053,25985357,6337295,car,09:38:07,00:00:00,112618,112618,2.538477e+06,1.152417e+06,2.538477e+06,1.152417e+06


In [55]:
trip_local_merged['distance'] = np.sqrt((trip_local_merged['start_x'] - trip_local_merged['end_x']) ** 2 +
                                (trip_local_merged['start_y'] - trip_local_merged['end_y']) ** 2)
# Ensure that trav_time is in timedelta format
trip_local_merged['trav_time'] = pd.to_timedelta(trip_local_merged['trav_time'])

average_duration = trip_local_merged['trav_time'].mean()  # convertir en minutes TO CHECK


# Now, calculate the average_speed
trip_local_merged['average_speed'] = (trip_local_merged['distance'] / 1000) / (trip_local_merged['trav_time'].dt.total_seconds() / 3600)

# print(trip_local_merged.head(5)) # There is duplicates via the dep time or mode !!!

average_distance = trip_local_merged['distance'].mean() / 1000  # convert to km
overall_average_speed = trip_local_merged['average_speed'].mean()

print(f"Average durations of trips in {LOCAL} : {average_duration}m")
print("Average Distance: {:.2f} km".format(average_distance))
print("Overall Average Speed: {:.2f} km/h".format(overall_average_speed))


         Unnamed: 0       id  mode  dep_time       trav_time start_link  \
147102     25983202  6336880   car  07:06:35 0 days 00:10:06     290992   
1880518    27935067  6740390  walk  08:35:31 0 days 00:26:13     290992   
147103     25983203  6336880   car  16:51:35 0 days 00:00:00     112618   
149052     25985356  6337295   car  08:38:07 0 days 00:00:00     112618   
149053     25985357  6337295   car  09:38:07 0 days 00:00:00     112618   

        end_link       start_x       start_y         end_x         end_y  \
147102    112618  2.539915e+06  1.152774e+06  2.538477e+06  1.152417e+06   
1880518   112618  2.539915e+06  1.152774e+06  2.538477e+06  1.152417e+06   
147103    112618  2.538477e+06  1.152417e+06  2.538477e+06  1.152417e+06   
149052    112618  2.538477e+06  1.152417e+06  2.538477e+06  1.152417e+06   
149053    112618  2.538477e+06  1.152417e+06  2.538477e+06  1.152417e+06   

            distance  average_speed  
147102   1481.858609       8.803120  
1880518  1481.85

In [8]:
work_facilities_df = activity_local_filtered[activity_local_filtered['type'] == 'work'][['id', 'facility', 'x', 'y']].drop_duplicates() # verifier via les homes
population_local_merged = population_local.merge(work_facilities_df, on='id', how='left')
population_local_merged.rename(columns={'facility': 'work_facility', 'x': 'work_facility_x', 'y': 'work_facility_y'}, inplace=True)

population_local_merged.head(1)

,id,age,home_x,home_y,local,work_facility,work_facility_x,work_facility_y
0,6336880,30,2539957.0,1152783.0,Lausanne,95509,2538446.0,1152412.0


In [34]:
# Dataframes for validations on a small numbers

n = 1 # nb of individuals
m = 10 # nb of activities availables

activity_local_filtered_sample = activity_local_filtered.sample(m)
population_local_merged_sample = population_local_merged.sample(n)

In [11]:
# DO NOT RUN BELOW : JOIN TO GET TRIP DISTANCE IS TOO HEAVY
# je pense que si je me penche moi meme dessus, je peux le resoudre

In [10]:
activity_local_filtered.to_csv(f'data_preprocessed/activity.csv', index=False)
trip_local.to_csv(f'data_preprocessed/trip.csv', index=False)
population_local_merged.to_csv(f'data_preprocessed/population.csv', index=False)